# SQL Assignment

In [0]:
import pandas as pd
import sqlite3

In [0]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls "/content/drive/My Drive/Colab Notebooks"

Db-IMDB-Assignment.db		Untitled0.ipynb  Untitled3.ipynb
SQL_Assignment_Reference.ipynb	Untitled1.ipynb  Untitled4.ipynb


In [0]:
conn = sqlite3.connect("/content/drive/My Drive/Colab Notebooks/Db-IMDB-Assignment.db")

## Sample Code

In [0]:
%%time
# Write your sql query below

query = """
        SELECT title AS 'Movie_Name'
        FROM Movie
        WHERE rating < 3
        
        """

q = pd.read_sql_query(query, conn)    
print(q)
q.head()

              Movie_Name
0             Mastizaade
1   Dragonball Evolution
2              Loveyatri
3                 Race 3
4                 Gunday
..                   ...
80          Bach ke Zara
81         Chudail Story
82         Suno Sasurjee
83           Veerabhadra
84       Impatient Vivek

[85 rows x 1 columns]
CPU times: user 96.2 ms, sys: 7.99 ms, total: 104 ms
Wall time: 109 ms


## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [0]:
%%time
# Write your sql query below
query = """
        SELECT p.Name,m.title,m.year FROM Movie as m,Person as p,M_Director as md WHERE ((m.year%4=0 AND m.year%100!=0) OR m.year%400=0) AND TRIM(m.MID)=TRIM(md.MID) AND TRIM(p.PID)=TRIM(md.PID) AND TRIM(md.MID) IN
        (SELECT TRIM(MID) FROM M_Genre WHERE TRIM(GID) IN 
        (SELECT TRIM(GID) FROM Genre WHERE Name LIKE '%Comedy%')
        )
        
        
        """

q1 = pd.read_sql_query(query, conn)    
print(q1)
q1.head()

                       Name                              title  year
0              Milap Zaveri                         Mastizaade  2016
1              Danny Leiner  Harold & Kumar Go to White Castle  2004
2            Anurag Kashyap                 Gangs of Wasseypur  2012
3              Frank Coraci        Around the World in 80 Days  2004
4             Griffin Dunne             The Accidental Husband  2008
..                      ...                                ...   ...
241   Siddharth Anand Kumar                        Let's Enjoy  2004
242         Amma Rajasekhar                            Sathyam  2008
243           Oliver Paulus                      Tandoori Love  2008
244             Raja Chanda                        Le Halua Le  2012
245        K.S. Prakash Rao                  Raja Aur Rangeeli  1996

[246 rows x 3 columns]
CPU times: user 2.28 s, sys: 9.31 ms, total: 2.29 s
Wall time: 2.31 s


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [0]:
%%time
# Write your sql query below

query = """
         SELECT Name FROM Person WHERE TRIM(PID) IN
         (
         SELECT TRIM(PID) FROM M_Cast WHERE TRIM(MID) IN
        (SELECT TRIM(MID) year FROM Movie WHERE TRIM(title)="Anand")
        )
        
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)
print(q2)
q2.head()

(17, 1)
                 Name
0    Amitabh Bachchan
1       Rajesh Khanna
2       Sumita Sanyal
3          Ramesh Deo
4           Seema Deo
5      Asit Kumar Sen
6          Dev Kishan
7        Atam Prakash
8       Lalita Kumari
9              Savita
10     Brahm Bhardwaj
11       Gurnam Singh
12       Lalita Pawar
13        Durga Khote
14         Dara Singh
15      Johnny Walker
16          Moolchand
CPU times: user 113 ms, sys: 26.6 ms, total: 139 ms
Wall time: 291 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [0]:
%%time
# Write your sql query below

query = """
        SELECT Name FROM Person WHERE TRIM(PID) in
        (SELECT TRIM(PID) FROM M_Cast WHERE TRIM(MID) IS NOT NULL AND TRIM(MID) IN
        (SELECT TRIM(MID) FROM Movie WHERE year<1970 OR year>1990) 
        )
        
        """

q3 = pd.read_sql_query(query, conn)    
print(q3.shape)
print(q3.head())

(29661, 1)
                    Name
0         Christian Bale
1         Cate Blanchett
2   Benedict Cumberbatch
3          Naomie Harris
4            Andy Serkis
CPU times: user 125 ms, sys: 9.66 ms, total: 135 ms
Wall time: 140 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [0]:
%%time
# Write your sql query below

query = """
        SELECT p.Name,COUNT(TRIM(md.MID)) AS No_of_movies FROM Person AS p,M_Director AS md WHERE TRIM(p.PID)=TRIM(md.PID) GROUP BY TRIM(p.Name)
         HAVING COUNT(TRIM(md.MID))>=10 ORDER BY COUNT(TRIM(md.MID)) DESC 
        
        """

q4 = pd.read_sql_query(query, conn)    
print(q4.shape)
print(q4)
q4.head()

(58, 2)
                         Name  No_of_movies
0                David Dhawan            39
1                Mahesh Bhatt            36
2                Priyadarshan            30
3             Ram Gopal Varma            30
4                Vikram Bhatt            29
5        Hrishikesh Mukherjee            27
6                 Yash Chopra            21
7             Basu Chatterjee            19
8              Shakti Samanta            19
9                Subhash Ghai            18
10   Abbas Alibhai Burmawalla            17
11          Rama Rao Tatineni            17
12              Shyam Benegal            17
13                     Gulzar            16
14             Manmohan Desai            16
15               Raj N. Sippy            16
16           Mahesh Manjrekar            15
17                 Raj Kanwar            15
18                Indra Kumar            14
19               Rahul Rawail            14
20                 Raj Khosla            14
21          Rajkumar San

## Q5.a --- For each year, count the number of movies in that year that had only female actors.

**SOLUTION 1**  

In [0]:
%%time
query = """
        SELECT M.YEAR, COUNT(M.MID) AS Number_of_movies_with_only_female_actors 
        FROM Movie AS M 
        WHERE M.MID IN 
        (SELECT X.MID 
        FROM (SELECT temp.MID AS MID, COUNT(temp.PID) AS TOTAL_CAST FROM M_Cast temp GROUP BY temp.MID) AS X  
        WHERE 
        X.TOTAL_CAST = (SELECT COUNT(A.PID)
        FROM M_Cast A, Person B 
        WHERE A.MID = X.MID AND B.PID = TRIM(A.PID) AND B.Gender = 'Female')
        )
        GROUP BY M.year
        
        """

q5a = pd.read_sql_query(query, conn)    
print(q5a.shape)
print(q5a)


(6, 2)
     year  Number_of_movies_with_only_female_actors
0    1939                                         1
1    1999                                         1
2    2000                                         1
3    2009                                         1
4    2012                                         1
5  I 2018                                         1
CPU times: user 40.5 s, sys: 2.05 s, total: 42.5 s
Wall time: 42.6 s


**SOLUTION 2**

In [0]:
%%time
# Write your sql query below
query = """
        SELECT m.year, COUNT(*) AS movies_with_only_female_actors FROM Movie m
                                join
                                (
                                    SElECT DISTINCT MID FROM M_Cast  
                                    WHERE MID NOT IN 
                                    (
                                        SELECT mc.MID FROM M_Cast mc 
                                        JOIN Person p ON p.pid = TRIM(mc.pid)
                                        WHERE p.Gender = 'Male'
                                    )
                                ) AS X ON X.MID = m.MID
                                GROUP BY m.year
        """
q5a = pd.read_sql_query(query, conn)    
print(q5a.shape)
print(q5a)

(7, 2)
     year  movies_with_only_female_actors
0    1939                               1
1    1999                               1
2    2000                               1
3    2009                               1
4    2012                               1
5    2018                               1
6  I 2018                               1
CPU times: user 170 ms, sys: 5.63 ms, total: 175 ms
Wall time: 180 ms


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [0]:
%%time
# Write your sql query below

query = """
          SELECT M.year,Y.Number_of_movies_with_only_female_actors,(Y.Number_of_movies_with_only_female_actors*100/COUNT(M.MID)*1.0) AS percentage_of_movies_with_only_female_actors,COUNT(M.MID) FROM
          (SELECT M.YEAR AS year, COUNT(M.MID) AS Number_of_movies_with_only_female_actors
          FROM Movie AS M 
          WHERE TRIM(M.MID) IN 
          (SELECT TRIM(X.MID) 
          FROM (SELECT TRIM(temp.MID) AS MID, COUNT(temp.PID) AS TOTAL_CAST FROM M_Cast temp GROUP BY TRIM(temp.MID)) AS X  
          WHERE X.TOTAL_CAST = (SELECT COUNT(TRIM(A.PID)) 
                                FROM M_Cast A, Person B 
                                WHERE TRIM(A.MID) = TRIM(X.MID) AND TRIM(B.PID) = TRIM(A.PID) AND B.Gender = 'Female')
          )
          GROUP BY M.year
          ) AS Y,Movie AS M 
          WHERE Y.year=M.year
          GROUP BY M.year
        """

q5b = pd.read_sql_query(query, conn)    
print(q5b.shape)
print(q5b)
q5b.head()

(6, 4)
     year  ...  COUNT(M.MID)
0    1939  ...             2
1    1999  ...            66
2    2000  ...            64
3    2009  ...           100
4    2012  ...           109
5  I 2018  ...            10

[6 rows x 4 columns]
CPU times: user 3min 51s, sys: 3.23 s, total: 3min 54s
Wall time: 3min 54s


In [0]:
q5b

,year,Number_of_movies_with_only_female_actors,percentage_of_movies_with_only_female_actors,COUNT(M.MID)
0,1939,1,50.0,2
1,1999,1,1.0,66
2,2000,1,1.0,64
3,2009,1,1.0,100
4,2012,1,0.0,109
5,I 2018,1,10.0,10


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [0]:
%%time
# Write your sql query below

query = """
        SELECT M.title,MAX(X.count_of_casts) AS cast_size
        FROM (SELECT MID , COUNT(DISTINCT(PID)) AS count_of_casts FROM M_Cast GROUP BY MID) AS X,Movie AS M
        WHERE TRIM(X.MID)=TRIM(M.MID)
        
        """

q6 = pd.read_sql_query(query, conn)    
print(q6.shape)
print(q6)


(1, 2)
           title  cast_size
0  Ocean's Eight        238
CPU times: user 2.09 s, sys: 0 ns, total: 2.09 s
Wall time: 2.1 s


## Q7 --- A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [0]:
%%time
# Write your sql query below

query = """
        SELECT y.year AS Decade_start, y.year + 9 AS decade_end,COUNT(*) AS Number_of__movies
        FROM (SELECT DISTINCT year FROM Movie) y join Movie m on m.year >= y.year and m.year < y.year + 10
        GROUP BY y.year
        ORDER BY COUNT(*) DESC
        LIMIT 1
        """

q7 = pd.read_sql_query(query, conn)    
print(q7.shape)
print(q7)

(1, 3)
  Decade_start  decade_end  Number_of__movies
0         2008        2017               1128
CPU times: user 89.7 ms, sys: 1.49 ms, total: 91.2 ms
Wall time: 94.3 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

**SOLUTION 1**

In [0]:
%%time
# Write your sql query below

query = """
          SELECT TRIM(X.PID) as PID,P.Name as actor_name,X.no_of_films_with_yash_chopra
          FROM (SELECT TRIM(PID) AS PID,COUNT(TRIM(MID)) AS no_of_films_with_yash_chopra FROM M_Cast WHERE TRIM(MID) IN
                 (SELECT TRIM(MID) FROM M_Director WHERE TRIM(PID) IN 
                 (SELECT TRIM(PID) FROM Person WHERE TRIM(Name)="Yash Chopra")) GROUP BY TRIM(PID)) AS X , Person AS p
          WHERE X.no_of_films_with_yash_chopra > 
            (SELECT MAX(no_of_films_with_other_director)
          FROM (SELECT TRIM(PID) AS PID,COUNT(TRIM(MID)) AS no_of_films_with_other_director FROM M_Cast AS mc WHERE TRIM(mc.PID)=TRIM(X.PID) AND TRIM(MID) IN
                 (SELECT TRIM(MID) FROM M_Director WHERE TRIM(PID) IN 
                 (SELECT TRIM(PID) FROM Person WHERE TRIM(Name)!="Yash Chopra")) GROUP BY TRIM(mc.PID))
              )
            AND TRIM(X.PID)=TRIM(p.PID)
           
        """

q8 = pd.read_sql_query(query, conn)    
print(q8.shape)
print(q8)
q8.head()

(3, 3)
         PID    actor_name  no_of_films_with_yash_chopra
0  nm0007181   Yash Chopra                             2
1  nm1767604   Ashok Verma                             2
2  nm3163800         Nazir                             2
CPU times: user 6.41 s, sys: 441 ms, total: 6.85 s
Wall time: 6.87 s


**SOLUTION 2**

In [0]:
%%time
result_9 = pd.read_sql_query('''
                                SELECT b.Actor_Name, b.Yash_Chopra_Directed_Movies 
                                FROM
                                (
                                    SELECT DISTINCT TRIM(p.pid) as Actor_Id, 
                                    TRIM(p.name) as Actor_Name,   
                                    COUNT(DISTINCT m.mid) as Non_Yash_Chopra_Directed_Movies 
                                    FROM Person p 
                                    JOIN M_Cast mc ON TRIM(mc.pid) = p.pid 
                                    JOIN Movie m ON m.mid = mc.mid 
                                    JOIN M_Director md ON md.mid = m.mid 
                                    JOIN Person p1 ON p1.pid = TRIM(md.pid)
                                    WHERE TRIM(p1.name) != 'Yash Chopra'
                                    GROUP BY TRIM(p.pid)
                                )a
                                LEFT JOIN
                                (
                                    SELECT DISTINCT TRIM(p.pid) as Actor_Id, 
                                    TRIM(p.name) as Actor_Name,  
                                    TRIM(p1.name) as Director_Name, 
                                    COUNT(DISTINCT m.mid) as Yash_Chopra_Directed_Movies 
                                    FROM Person p 
                                    JOIN M_Cast mc ON TRIM(mc.pid) = p.pid 
                                    JOIN Movie m ON m.mid = mc.mid 
                                    JOIN M_Director md ON md.mid = m.mid 
                                    JOIN Person p1 ON p1.pid = TRIM(md.pid)
                                    WHERE TRIM(p1.name) = 'Yash Chopra'
                                    GROUP BY TRIM(p.pid)
                                )b ON a.Actor_Id = b.Actor_Id
                                WHERE b.Yash_Chopra_Directed_Movies > a.Non_Yash_Chopra_Directed_Movies
                                
                                
                            ''', conn) 
print(result_9)

    Actor_Name  Yash_Chopra_Directed_Movies
0  Yash Chopra                            2
1  Ashok Verma                            2
2        Nazir                            2
CPU times: user 948 ms, sys: 22.7 ms, total: 971 ms
Wall time: 980 ms


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [0]:
%%time
# Write your sql query below

query = """
        WITH 
    SHAHRUKH_0 AS
    (
        SELECT
            TRIM(P.PID) PID
        FROM
            Person P
        WHERE
            TRIM(P.Name) like '%Shahrukh%'
    ),
    SHAHRUKH_1_MOVIES AS
    (
        SELECT
            DISTINCT
            TRIM(MC.MID) MID,
            S0.PID
        FROM
            M_Cast MC,
            SHAHRUKH_0 S0
        WHERE
            TRIM(MC.PID) = S0.PID
    ),
    SHAHRUKH_1_ACTORS AS
    (
        SELECT
            DISTINCT
            TRIM(MC.PID) PID
        FROM
            M_Cast MC,
            SHAHRUKH_1_MOVIES S1M
        WHERE
            TRIM(MC.MID) = S1M.MID AND
            TRIM(MC.PID) <> S1M.PID
    ),
    SHAHRUKH_2_MOVIES AS
    (
        SELECT
            DISTINCT
            TRIM(MC.MID) MID,
            S1A.PID
        FROM
            M_Cast MC,
            SHAHRUKH_1_ACTORS S1A
        WHERE
            TRIM(MC.PID) = S1A.PID
    )
    SELECT
        DISTINCT
        TRIM(MC.PID) PID,
        TRIM(P.Name) ACTOR_NAME
    FROM
        Person P,
        M_Cast MC,
        SHAHRUKH_2_MOVIES S2M
    WHERE
            TRIM(MC.PID) = TRIM(P.PID) AND
            TRIM(MC.MID) = S2M.MID AND
            TRIM(MC.PID) <> S2M.PID;
        
        """

q9 = pd.read_sql_query(query, conn)    
print(q9.shape)
q9.head()

(15911, 2)
CPU times: user 9min 6s, sys: 28.9 s, total: 9min 35s
Wall time: 9min 36s


In [0]:
print(q9)

             PID             ACTOR_NAME
0      nm2951768           Freida Pinto
1      nm6467532  Caroline Christl Long
2      nm6071249          Rajeev Pahuja
3      nm3491108      Michelle Santiago
4      nm7509518         Jandre le Roux
...          ...                    ...
15906  nm3099317           Dhruv Shetty
15907  nm2371614        Hayley Cleghorn
15908  nm2675737        Nirvasha Jithoo
15909  nm2370589         Kamal Maharshi
15910  nm1866356           Mohini Manik

[15911 rows x 2 columns]
